<a href="https://colab.research.google.com/github/karou1182001/NLPAssignments/blob/main/Assignment4/NLPAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Assignment 4**

The goal of this assignment is to:
1. Familiarize yourself with how to set up a cloud-based environment (e.g., Colab) to train and evaluate
LLMs using open-source tools.
2. Learn how to fine-tune a pre-trained large language model on a custom text dataset.
3. Conduct inference with the fine-tuned model and evaluate its outputs.
Prerequisites

The goal was to teach the model how to write in a more specific style by adjusting it to new training data.


##1. Set Up Your Environment

Make sure to enable a GPU runtime for faster training

In [ ]:
import torch

# Verifying if GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

x = torch.rand(1).to(device)
print(f"Tensor in {device}: {x}")

Running on: cuda
GPU: Tesla T4
Tensor in cuda: tensor([0.7674], device='cuda:0')


Install or upgrade your necessary libraries (if needed)

In [ ]:
!pip install --quiet transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

Import packages

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset


print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.6.0+cu124
CUDA available: True


## 2. Choose a Pretrained Model

 We will use a smaller model so that fine-tuning can be done within a reasonable time
and within the memory constraints of free GPU instances.
I used distilgpt2, a small version of GPT-2.

Them, we loaded a tokenizer to turn text into tokens (numbers).

and also defined a padding token to avoid errors.

We use a pretrained model as the base. Fine-tuning means we start from this general knowledge and teach it something more specific.


In [ ]:
model_name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  #We added this part to avoid error
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer)) #added


model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In this part, we want to check the results of the model without the fine tuned

In [ ]:

#prompt = "The invention of the printing press in the 15th century had a significant impact on"
prompt = "I really loved the acting in this movie because"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)


original_output = model.generate(
    input_ids=inputs["input_ids"],
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

original_text = tokenizer.decode(original_output[0], skip_special_tokens=True)

print("Text without fine tuned:")
print(original_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Text without fine tuned:
I really loved the acting in this movie because I thought it would be fun to do it. I think it was a great experience for me. It was really fun and I'm really excited for it to be a part of the movie.




## 3. Prepare a Text Dataset

I selected option A:
 Load a built-in dataset from Hugging Face (e.g., wikitext)

I used the public dataset recommended by the Homeowrk: Wikitext-2.

This dataset contains text similar to Wikipedia articles and the model will learn the writing style and vocabulary from this dataset

In [ ]:
#Load dataset
#dataset = load_dataset("wikitext", "wikitext-2-raw-v1") #Used by the guide
dataset = load_dataset("imdb") #The one I decided to use

dataset

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Split the dataset into training and validation sets if not already split

In [ ]:
#train_data = dataset["train"]
#val_data = dataset["validation"]

dataset_split = dataset["train"].train_test_split(test_size=0.1)
train_data = dataset_split["train"]
val_data = dataset_split["test"]

#To validate we have the data

# Search for the first not empty text
for example in train_data:
    if example["text"].strip():  # If it is not empty
        print("Train example:\n", example["text"])
        break

# Search for the first not empty text
for example in val_data:
    if example["text"].strip():
        print("\nVal example:\n", example["text"])
        break



Train example:
 The Sopranos stands out as an airtight, dynamic exploration of American life, and how the American experience is shaped and defined by money. By setting the story in the milieu of the underworld, David Chase eliminates all barriers to a grunt, low to the ground and outright mean deconstruction of the post-modern era. <br /><br />Every character represents a facet of American industry. Tony Soprano exemplifies the beleaguered working stiff, torn between familial duty and a need to keep his "business" on an even keel. The convergence of these two things is the imperative that keeps the story moving forward. The characters of Christopher, Paulie, and Bobby reflect the loyal - but self-serving underlings present in every enterprise, who are trusted out of necessity rather than merit. With the character of Ralph, Joe Pantoliano essays a brilliant interpretation of the charismatic psychopath, a twisted businessman who's flourishes of violence are tragically outweighed by his 

## 4. Preprocess the Data

Define a preprocessing function to tokenize your text data:(converted words to numbers).

Created labels to help the model learn what to predict.

Formatted the data for training.

In [ ]:
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_val = val_data.map(tokenize_function, batched=True)

# Remove columns other than input_ids and attention_mask
tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


#To see if worked:
print(tokenized_train[0])


#For a better time of training
small_train = tokenized_train.select(range(5000))
small_test = tokenized_val.select(range(1000))


Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

{'input_ids': tensor([  464, 35643,  2596,   418,  6296,   503,   355,   281,  1633, 33464,
           11,  8925, 13936,   286,  1605,  1204,    11,   290,   703,   262,
         1605,  1998,   318, 14292,   290,  5447,   416,  1637,    13,  2750,
         4634,   262,  1621,   287,   262,  1465, 22304,   286,   262, 42494,
           11,  3271, 16346, 32311,   477, 14725,   284,   257, 45836,    11,
         1877,   284,   262,  2323,   290, 15828,  1612, 37431, 15019,   286,
          262,  1281,    12, 23922,  6980,    13,  1279,  1671,  1220,  6927,
         1671, 11037,  6109,  2095,  6870,   257, 39144,   286,  1605,  2831,
           13,  8832, 35643, 35823, 21433,  6945,   262,   307, 19316,   445,
         1762, 15175,    11, 12445,  1022, 48376,  7077,   290,   257,   761,
          284,  1394,   465,   366, 22680,     1,   319,   281,   772,   885,
          417,    13,   383, 40826,   286,   777,   734,  1243,   318,   262,
        23602,   326,  7622,   262,  1621,  3867, 

## 5. Fine-Tune the Model

Use the Trainer API to set up your fine-tuning pipeline:

This is where the model actually learns from the dataset. We fine-tune it so it starts writing more like our training data.

In [ ]:
from transformers import TrainingArguments


training_args = TrainingArguments(
    output_dir="./finetuned_llm",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=7,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
    report_to="none" #Not connect to any account
)

'''
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_train,
  eval_dataset=tokenized_val,
)
'''

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,   # Subset instead of the full dataset
    eval_dataset=small_test      # Subset for validation- test
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.835200,3.731128
2,3.640800,3.728120
3,3.434400,3.743970
4,3.306100,3.774298
5,3.234000,3.800623
6,3.060800,3.836709
7,3.050800,3.851266


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=17500, training_loss=3.341697420828683, metrics={'train_runtime': 1392.383, 'train_samples_per_second': 25.137, 'train_steps_per_second': 12.568, 'total_flos': 1143173283840000.0, 'train_loss': 3.341697420828683, 'epoch': 7.0})

During the 7 training epochs, the training loss went down from 3.83 to 3.05. This means the model was learning from the training data and improving over time. The validation loss stayed almost the same, but it started to go up a little after the third or fourth epoch. This could mean that the model was starting to overfit. However, the model still produced good quality text, so the training worked well overall.

## 6. Evaluate the Fine-Tuned Model

This step lets us see what the model learned and how it writes after training.
We can compare the results before fine-tuned, and the results after fine tuned.
In this part, I decided to play a little bit with the model changing parameters as the temperature and the top: temperature=0.8 and top_p=0.9. This setup is meant to keep the output creative but still coherent. A temperature of 0.8 lowers the randomness a bit, helping the model stay more focused in its responses. At the same time, using top_p=0.9 allows the model to sample from the most likely words, while still leaving room for some variation. Overall, I expect this combination to generate responses that are natural, interesting, and not too repetitive.

In [ ]:
#prompt = "The invention of the printing press in the 15th century had a significant impact on"
prompt = "I really loved the acting in this movie because"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

# Generate text
output_ids = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2,
    temperature=0.8,
    top_p=0.9,
    early_stopping=True
)

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Prompt:", prompt)
print("Generated text with fine tuned:\n", generated_text)
print("Generated text without fine tuned:\n", original_text)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: I really loved the acting in this movie because
Generated text with fine tuned:
 I really loved the acting in this movie because it was so well done. The acting was great, the story was believable, and the characters were believable. It was a very enjoyable movie. I would recommend it to anyone who is a fan of the
Generated text without fine tuned:
 I really loved the acting in this movie because I thought it would be fun to do it. I think it was a great experience for me. It was really fun and I'm really excited for it to be a part of the movie.




I used the prompt "I really loved the acting in this movie because" to compare outputs before and after fine tuning. The fine tuned model gave a much more coherent and realistic response. It described the acting, story, and characters in a way that sounds like a proper review, and even included a recommendation, which is very relevant for IMDb-= style text.
In contrast, the original model produced a vague and repetitive sentence that didn’t stay on topic. It even mentioned "being in the movie," which doesn’t fit the original prompt. This shows that fine tuning helped the model understand the tone and structure of movie reviews better.
